Training a 2D bbox to 2D segmentation model
---

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import os
from evaluation.eval_utils import get_seg_bbox
from pathlib import Path
from tqdm import tqdm

import nrrd
import SimpleITK as sitk
from utils.plot import transparent_cmap

In [ ]:
# Dataset
data_folder = Path("/media/liushifeng/KINGSTON/ULS Jan 2025/ULS23/novel_data/ULS23_DeepLesion3D")
seg_folder = data_folder / "labels"
ct_folder = data_folder / "images"

with open(data_folder / "train.txt", "r") as f:
    train_names = [x.strip() for x in f.readlines()]
with open(data_folder / "val.txt", "r") as f:
    val_names = [x.strip() for x in f.readlines()]

filenames = [x for x in os.listdir(seg_folder) if ".zip" not in x]
seg_paths = {x: seg_folder / x for x in filenames}
ct_paths = {x: ct_folder / x for x in filenames}

In [ ]:
ignored_train_samples = [x.strip() for x in
    """
    000148_04_01_034_lesion_01
    003287_01_01_188_lesion_01
    003931_01_01_078_lesion_01
    000026_06_01_257_lesion_01
    000346_01_01_085_lesion_01
    000215_05_01_096_lesion_01
    001354_04_02_305_lesion_01
    001564_02_02_513_lesion_01
    """.split("\n") if x.strip()]

In [ ]:
# for plotting histograms
bins = [int(x) for x in np.arange(-1000, 2001, 50)] + [2050] # Define HU bins
hu_counter = Counter()  # Initialize the counter

for every lesion, based on filename:
- input
    - crop (32X * 32Y)
    - rectangle patch or border of the segment (32X * 32Y)
    - full slice as context (fixed size e.g. 256 * 256)
- output
    - seg (32X * 32Y) **Blur for soft mask?**

In [ ]:
seg, _ = nrrd.read("/media/liushifeng/KINGSTON/ULS DL3D 2D dataset/masks/002779_01_01_183_lesion_01_slice61_seg")

In [ ]:
from train_utils import crop_from_img

output_folder = Path("/media/liushifeng/KINGSTON/ULS DL3D 2D dataset/")

plot = 0
save = 0

for filename in list(seg_paths.keys()):
    if filename.split(".")[0] in ignored_train_samples:
        print("skipped:", filename.split(".")[0])
        continue

    lesion_name = filename.split(".")[0]

    # load arrays
    ct = sitk.ReadImage(ct_paths[filename])
    ct_array = sitk.GetArrayFromImage(ct)
    seg = sitk.ReadImage(seg_paths[filename])
    seg_array = sitk.GetArrayFromImage(seg)

    # get slices where there are segmentations
    seg_slice_indices = np.where(seg_array.any(axis=(1,2)) > 0)[0]

    for i in seg_slice_indices:
        ct_slice = ct_array[i]
        seg_slice = seg_array[i]

        crop_bbox = get_seg_bbox(seg_slice)

        if save:
            print(f"saving {lesion_name}_slice{i}")
            # train_or_val = "train" if lesion_name in train_names else "val"
            # save_folder = output_folder / train_or_val
            # save_folder.mkdir(parents=True, exist_ok=True)
            np.save(str(output_folder / "images" / f"{lesion_name}_slice{i}.npy"), ct_slice)
            # nrrd.write(str(output_folder / "slices" / f"{lesion_name}_slice{i}_seg"), seg_slice)

        if plot:
            vmin, vmax = -200, 200
            plt.imshow(ct_slice, vmin=vmin, vmax=vmax, cmap='gray')
            plt.show()
            plt.imshow(seg_slice)
            plt.show()

            # crops areas for plotting
            margin = 2
            ct_crop = crop_from_img(ct_slice, crop_bbox, margin, AIR_VALUE)
            seg_crop = crop_from_img(seg_slice, crop_bbox, margin, 0)

        # hu_counter.update(np.digitize(ct_crop.flatten(), bins))
        # hu_counter.update(np.digitize(ct_crop[seg_crop == 1], bins))

In [ ]:
# if save:
#     # save training data set
#     train_or_val = "train" if lesion_name in train_names else "val"
#     save_folder = output_folder / train_or_val
#     save_folder.mkdir(parents=True, exist_ok=True)
#
#     np.save(save_folder / f"{lesion_name}_slice{i}_crop", np.array(ct_crop, dtype=np.int16))
#     np.save(save_folder / f"{lesion_name}_slice{i}_cropseg", np.array(seg_crop, dtype=np.uint8))
#     np.save(save_folder / f"{lesion_name}_slice{i}_slice", np.array(ct_slice, dtype=np.int16))
#
# if plot:
#     _, axes = plt.subplots(1, 2, figsize=(4, 2))
#
#     # plot the crop and segment
#     vmin, vmax = -200, 200
#     axes[0].imshow(ct_crop, vmin=vmin, vmax=vmax, cmap='gray')
#     axes[1].imshow(ct_crop, vmin=vmin, vmax=vmax, cmap='gray')
#     axes[1].imshow(seg_crop, cmap=transparent_cmap("blue"), alpha=0.4)
#     for ax in axes:
#         ax.axis("off")
#     plt.show()
#
#     # plot the entire slice
#     _, axes = plt.subplots(1, 2, figsize=(10, 5))
#     axes[0].imshow(ct_slice, cmap='gray')
#     axes[1].imshow(ct_slice, cmap='gray')
#     axes[1].imshow(seg_slice, cmap=transparent_cmap("blue"), alpha=0.4)
#     for ax in axes:
#         ax.axis("off")
#     plt.show()

In [ ]:
plt.figure(figsize=(6,2.5));
plt.bar([bins[int(x)] for x in hu_counter.keys()], hu_counter.values(), width=50);

In [ ]:
plt.figure(figsize=(6,2.5));
plt.bar([bins[int(x)] for x in hu_counter.keys()], hu_counter.values(), width=50);

In [ ]:
plt.imshow(ct_slice, cmap='gray'); plt.axis("off");
plt.imshow(seg_slice, cmap='jet', alpha=0.5); plt.axis("off");

## Load data

In [ ]:
data_dir = Path("/media/liushifeng/KINGSTON/ULS DL3D 2D dataset")

In [ ]:
import os
import numpy as np
from torch.utils.data import DataLoader
from dataset import SegmentationDataset

image_dir = data_dir / "images"
label_dir = data_dir / "masks"
all_images = sorted(os.listdir(image_dir))

# random.shuffle(all_images)  # Shuffle before splitting
# val_size = int(0.25 * len(all_images))
# val_images = all_images[:val_size]
# train_images = all_images[val_size:]

val_images = [x for x in all_images if x.split("_slice")[0] in val_names]
train_images = [x for x in all_images if x.split("_slice")[0] in train_names]

In [ ]:
from transforms import train_transform, val_transform

image_names = None
batch_size = 1
workers = 20
train_dataset = SegmentationDataset(image_dir, label_dir, train_images, train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

val_dataset = SegmentationDataset(image_dir, label_dir, val_images, val_transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)

# Visualizing data loader
data = next(iter(val_loader))
plt.figure(figsize=(3,3))
plt.imshow(data['image'].squeeze(), cmap='gray')
plt.imshow(data['mask'].squeeze(), cmap=transparent_cmap("blue"), alpha=0.5)
plt.imshow(data['box_mask'].squeeze(), cmap='Greens', alpha=0.3)
# plt.axis('off');
plt.show()

## Train Model

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)
import torch
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping

torch.set_float32_matmul_precision('medium')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from model import UNet

unet = UNet()
early_stopping = EarlyStopping('val_loss', verbose=True)
trainer = L.Trainer(
    accumulate_grad_batches=32,
    max_epochs=20,
    callbacks=[early_stopping]
)
trainer.fit(unet, train_loader, val_loader)

In [ ]:
trainer.early_stopping_callback.stopped_epoch

In [ ]:
# visualize data and model output
from torchmetrics.segmentation import DiceScore

dice = DiceScore(1)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

res = []
n = 0
for i, data in tqdm(enumerate(val_loader)):
# data = next(iter(val_loader))
    img = data['input'].squeeze()[0]
    out_raw = unet.model.to(device)(data['input'].to(device)).detach().cpu().squeeze()
    out = out_raw.clone()
    out[out < 0.5] = 0

    gt = data['mask'].bool().squeeze()
    pred = out.bool().squeeze()

    tp = gt & pred
    fp = ~gt & pred
    fn = gt & ~pred

    dice_score = dice(pred.unsqueeze(0), gt.unsqueeze(0))
    dice_scores.append(dice_score.item())

    vol_sim = 1 - abs(pred.sum() - gt.sum()) / gt.sum()
    vol_scores.append(vol_sim.item())

    res.append({"dice": dice_score.item(), "vs": vol_sim.item()})
    # if fp.sum() + fn.sum() > max(70, (0.6 * tp.sum())):
    if False:
        print(dice_score)
        fig, ax = plt.subplots(1, 3, figsize=(6,2))
        ax[0].imshow(img, cmap='gray')

        ax[1].imshow(img, cmap='gray')
        ax[1].imshow(out * tp, cmap=transparent_cmap("green"), alpha=0.4)
        ax[1].imshow(out * fp, cmap=transparent_cmap("red"), alpha=0.4)
        ax[1].imshow(fn, cmap=transparent_cmap("blueviolet"), alpha=0.4)

        ax[2].imshow(out, vmin=0, vmax=1, cmap="summer_r", alpha=1)

        [a.axis('off') for a in ax]
        plt.show()
        n += 1

    if n > 1e10:
        break


In [ ]:
import pandas as pd

pd.DataFrame(res).describe()